# Initial Cleaning

In this notebook, I do the inital cleaning of the training data to prep it for EDA and initial feature engineering

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
df = pd.read_csv('./datasets/train.csv')

In [4]:
df.shape

(2051, 81)

In [5]:
df.isnull().sum()

Id                0
PID               0
MS SubClass       0
MS Zoning         0
Lot Frontage    330
               ... 
Misc Val          0
Mo Sold           0
Yr Sold           0
Sale Type         0
SalePrice         0
Length: 81, dtype: int64

Because there are so many columns, I wanted to identify only the columns that contain null values. I experimented and wasn't having any luck, so I did some research: this [stackoverflow](https://stackoverflow.com/questions/53137100/filter-pandas-dataframe-columns-with-null-data) showed me the approach I used below.

In [6]:
df.columns[df.isna().any()].tolist()

['Lot Frontage',
 'Alley',
 'Mas Vnr Type',
 'Mas Vnr Area',
 'Bsmt Qual',
 'Bsmt Cond',
 'Bsmt Exposure',
 'BsmtFin Type 1',
 'BsmtFin SF 1',
 'BsmtFin Type 2',
 'BsmtFin SF 2',
 'Bsmt Unf SF',
 'Total Bsmt SF',
 'Bsmt Full Bath',
 'Bsmt Half Bath',
 'Fireplace Qu',
 'Garage Type',
 'Garage Yr Blt',
 'Garage Finish',
 'Garage Cars',
 'Garage Area',
 'Garage Qual',
 'Garage Cond',
 'Pool QC',
 'Fence',
 'Misc Feature']

In [7]:
df.columns = df.columns.str.replace(' ', '_').str.lower()

In [8]:
df.head(2)

,id,pid,ms_subclass,ms_zoning,lot_frontage,lot_area,street,alley,lot_shape,land_contour,...,screen_porch,pool_area,pool_qc,fence,misc_feature,misc_val,mo_sold,yr_sold,sale_type,saleprice
0,109,533352170,60,RL,NaN,13517,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,3,2010,WD,130500
1,544,531379050,60,RL,43.0,11492,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,4,2009,WD,220000


### Sanity Check From EDA + Further Cleaning Notebook

I'm returning to this point to be certain that I didn't accidentally create any of the oddities I'm finding during EDA.

In [9]:
df['street'].value_counts()

Pave    2044
Grvl       7
Name: street, dtype: int64

## Data Dictionary

I reference the [data dictionary](http://jse.amstat.org/v19n3/decock/DataDocumentation.txt) a lot in the following.

### Alley - NaN --> 'None'

In [10]:
df['alley'].value_counts(dropna = False)

NaN     1911
Grvl      85
Pave      55
Name: alley, dtype: int64

It appears that NaN means there is no alley. I'm going to replace NaN with 'None' for now. 

In [11]:
df['alley'] = df['alley'].fillna('None')

df['alley'].value_counts(dropna = False)

None    1911
Grvl      85
Pave      55
Name: alley, dtype: int64

### Basements!

I created this for-loop below to examine how many NAs existed in each basement column, to help me decide how to deal with them, since there's presumably at least some relationship betwen them. Rather than copying the code over and over again, I also returned to the for-loop to use as my sanity check as I cleaned the basement columns.

In [12]:
bsmt_list = df.columns[df.columns.str.contains('bsmt')].tolist()

bsmt_list

['bsmt_qual',
 'bsmt_cond',
 'bsmt_exposure',
 'bsmtfin_type_1',
 'bsmtfin_sf_1',
 'bsmtfin_type_2',
 'bsmtfin_sf_2',
 'bsmt_unf_sf',
 'total_bsmt_sf',
 'bsmt_full_bath',
 'bsmt_half_bath']

In [13]:
for i in bsmt_list:
    print(f'{i} has {df[i].isnull().sum()}')

bsmt_qual has 55
bsmt_cond has 55
bsmt_exposure has 58
bsmtfin_type_1 has 55
bsmtfin_sf_1 has 1
bsmtfin_type_2 has 56
bsmtfin_sf_2 has 1
bsmt_unf_sf has 1
total_bsmt_sf has 1
bsmt_full_bath has 2
bsmt_half_bath has 2


I was surprised by the variation on the number of NaNs. Knowing that some of the NaNs actually represent "No Basement", I thought there'd be a pretty consistent number.

I decided to start by looking at the smaller values to see what's going on to start.

### Basement Full Bath and Half Bath

In [14]:
df[df['bsmt_full_bath'].isna()][bsmt_list]

,bsmt_qual,bsmt_cond,bsmt_exposure,bsmtfin_type_1,bsmtfin_sf_1,bsmtfin_type_2,bsmtfin_sf_2,bsmt_unf_sf,total_bsmt_sf,bsmt_full_bath,bsmt_half_bath
616,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0,0.0,NaN,NaN
1327,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
df['bsmt_full_bath'].value_counts(dropna = False)

0.0    1200
1.0     824
2.0      23
NaN       2
3.0       2
Name: bsmt_full_bath, dtype: int64

I did the above two lines to get a sense of what was going on with these instances. It appears these NaNs should be 0.0, reflecting no basement. The same appears to be true for the half baths, which I checked below.

In [16]:
df['bsmt_full_bath'] = df['bsmt_full_bath'].fillna(0.0)

In [17]:
df['bsmt_full_bath'].value_counts(dropna = False)

0.0    1202
1.0     824
2.0      23
3.0       2
Name: bsmt_full_bath, dtype: int64

In [18]:
df[df['bsmt_half_bath'].isna()][bsmt_list]

,bsmt_qual,bsmt_cond,bsmt_exposure,bsmtfin_type_1,bsmtfin_sf_1,bsmtfin_type_2,bsmtfin_sf_2,bsmt_unf_sf,total_bsmt_sf,bsmt_full_bath,bsmt_half_bath
616,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0,0.0,0.0,NaN
1327,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN


In [19]:
df['bsmt_half_bath'] = df['bsmt_half_bath'].fillna(0.0)

In [20]:
df['bsmt_half_bath'].value_counts(dropna = False)

0.0    1925
1.0     122
2.0       4
Name: bsmt_half_bath, dtype: int64

### Basement Sq Ft, Finished (both types) & Unfinished, Total

In [21]:
df[df['bsmt_unf_sf'].isna()][bsmt_list]

,bsmt_qual,bsmt_cond,bsmt_exposure,bsmtfin_type_1,bsmtfin_sf_1,bsmtfin_type_2,bsmtfin_sf_2,bsmt_unf_sf,total_bsmt_sf,bsmt_full_bath,bsmt_half_bath
1327,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0


Having looked at values in other instances, it appears this also should be 0.0, for no basement, as well.

In [22]:
df['bsmt_unf_sf'] = df['bsmt_unf_sf'].fillna(0.0)
df['bsmt_unf_sf'].isnull().sum()

0

In [23]:
df[df['bsmtfin_sf_1'].isnull()][bsmt_list]

,bsmt_qual,bsmt_cond,bsmt_exposure,bsmtfin_type_1,bsmtfin_sf_1,bsmtfin_type_2,bsmtfin_sf_2,bsmt_unf_sf,total_bsmt_sf,bsmt_full_bath,bsmt_half_bath
1327,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0


In [24]:
df['bsmtfin_sf_1'] = df['bsmtfin_sf_1'].fillna(0.0)
df['bsmtfin_sf_1'].isnull().sum()

0

In [25]:
df[df['bsmtfin_sf_2'].isnull()][bsmt_list]

,bsmt_qual,bsmt_cond,bsmt_exposure,bsmtfin_type_1,bsmtfin_sf_1,bsmtfin_type_2,bsmtfin_sf_2,bsmt_unf_sf,total_bsmt_sf,bsmt_full_bath,bsmt_half_bath
1327,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,0.0,0.0


In [26]:
df['bsmtfin_sf_2'] = df['bsmtfin_sf_2'].fillna(0.0)
df['bsmtfin_sf_2'].isnull().sum()

0

In [27]:
df[df['total_bsmt_sf'].isnull()][bsmt_list]

,bsmt_qual,bsmt_cond,bsmt_exposure,bsmtfin_type_1,bsmtfin_sf_1,bsmtfin_type_2,bsmtfin_sf_2,bsmt_unf_sf,total_bsmt_sf,bsmt_full_bath,bsmt_half_bath
1327,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0,NaN,0.0,0.0


In [28]:
df['total_bsmt_sf'] = df['total_bsmt_sf'].fillna(0.0)
df['total_bsmt_sf'].isnull().sum()

0

### Basement Quality

The dictionary specifies that 'Na' is 'No Basement', so I'm going to cast it as such. This appears to bet true for the categorical basement elements.

In [29]:
df['bsmt_qual'].value_counts(dropna = False)

TA     887
Gd     864
Ex     184
Fa      60
NaN     55
Po       1
Name: bsmt_qual, dtype: int64

In [30]:
df['bsmt_qual'] = df['bsmt_qual'].fillna('No Basement')

In [31]:
df['bsmt_qual'].value_counts(dropna = False)

TA             887
Gd             864
Ex             184
Fa              60
No Basement     55
Po               1
Name: bsmt_qual, dtype: int64

### Basement Condition

The data dictionary says that 'Na' means None, so I'm making that explicit.

In [32]:
df['bsmt_cond'] = df['bsmt_cond'].fillna('No Basement')
df['bsmt_cond'].isnull().sum()

0

In [33]:
df['bsmt_cond'].value_counts()

TA             1834
Gd               89
Fa               65
No Basement      55
Po                5
Ex                3
Name: bsmt_cond, dtype: int64

### Basement Exposure
Dictionary says 'Na' means 'No Basement', so making that explicit.

In [34]:
df['bsmt_exposure'] = df['bsmt_exposure'].fillna('No Basement')
df['bsmt_exposure'].isna().sum()

0

In [35]:
df['bsmt_exposure'].value_counts()

No             1339
Av              288
Gd              203
Mn              163
No Basement      58
Name: bsmt_exposure, dtype: int64

### Basement Finish Type 1 and 2
The dictionary sas 'Na' means 'No Basement', so making that explicit. There's one more Finish Type 2 than Type 1, so I looked into that before making the change to 'No Basement'.

In [36]:
df['bsmtfin_type_1'] = df['bsmtfin_type_1'].fillna('No Basement')
df['bsmtfin_type_1'].value_counts()

GLQ            615
Unf            603
ALQ            293
BLQ            200
Rec            183
LwQ            102
No Basement     55
Name: bsmtfin_type_1, dtype: int64

In [37]:
df[(df['bsmtfin_type_2'].isnull()) & (df['bsmtfin_type_1'] != 'No Basement')][bsmt_list]

,bsmt_qual,bsmt_cond,bsmt_exposure,bsmtfin_type_1,bsmtfin_sf_1,bsmtfin_type_2,bsmtfin_sf_2,bsmt_unf_sf,total_bsmt_sf,bsmt_full_bath,bsmt_half_bath
1147,Gd,TA,No,GLQ,1124.0,NaN,479.0,1603.0,3206.0,1.0,0.0


In the original data frame there's one instance that has a null 'basmtfin_type_2' that clearly has a basement based on the other entries. Since I have no way of speculating what type that should be, I'm deleting this instance.

In [38]:
df.drop(index = df[(df['bsmtfin_type_2'].isnull()) & (df['bsmtfin_type_1'] != 'No Basement')].index, inplace = True)

In [39]:
df[(df['bsmtfin_type_2'].isnull()) & (df['bsmtfin_type_1'] != 'No Basement')][bsmt_list]

,bsmt_qual,bsmt_cond,bsmt_exposure,bsmtfin_type_1,bsmtfin_sf_1,bsmtfin_type_2,bsmtfin_sf_2,bsmt_unf_sf,total_bsmt_sf,bsmt_full_bath,bsmt_half_bath


Now making the change to 'No Basement' on Finish Type 2

In [40]:
df['bsmtfin_type_2'] = df['bsmtfin_type_2'].fillna('No Basement')
df['bsmtfin_type_2'].value_counts()

Unf            1749
Rec              80
LwQ              60
No Basement      55
BLQ              48
ALQ              35
GLQ              23
Name: bsmtfin_type_2, dtype: int64

Checking in on which columns still have Na's

In [41]:
df.columns[df.isnull().any()].tolist()

['lot_frontage',
 'mas_vnr_type',
 'mas_vnr_area',
 'fireplace_qu',
 'garage_type',
 'garage_yr_blt',
 'garage_finish',
 'garage_cars',
 'garage_area',
 'garage_qual',
 'garage_cond',
 'pool_qc',
 'fence',
 'misc_feature']

### Garage Columns

In [42]:
garage_list = df.columns[df.columns.str.contains('garage')].tolist()
garage_list

['garage_type',
 'garage_yr_blt',
 'garage_finish',
 'garage_cars',
 'garage_area',
 'garage_qual',
 'garage_cond']

Again, I recycled the below for-loop as a final sanity check, returning to this line repeatedly as I worked, rather than rewriting the loop for every step I took.

In [43]:
for i in garage_list:
    print(f'{i} has {df[i].isnull().sum()} nulls')

garage_type has 113 nulls
garage_yr_blt has 114 nulls
garage_finish has 114 nulls
garage_cars has 1 nulls
garage_area has 1 nulls
garage_qual has 114 nulls
garage_cond has 114 nulls


I did a search to find the not null method for the following, which turned up the [pandas documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.notnull.html).

In [44]:
df[(df['garage_yr_blt'].isnull()) & (df['garage_type'].notnull())][garage_list]

,garage_type,garage_yr_blt,garage_finish,garage_cars,garage_area,garage_qual,garage_cond
1712,Detchd,NaN,NaN,NaN,NaN,NaN,NaN


A garage_type is listed for this one instance, but NaN in a number of the other columns means 'No Garage' according to the dictionary, so I'm going to operate on the assumption that the garage type is in error, here and change it to 'No Garage'

In [45]:
df.reset_index()

,index,id,pid,ms_subclass,ms_zoning,lot_frontage,lot_area,street,alley,lot_shape,...,screen_porch,pool_area,pool_qc,fence,misc_feature,misc_val,mo_sold,yr_sold,sale_type,saleprice
0,0,109,533352170,60,RL,NaN,13517,Pave,None,IR1,...,0,0,NaN,NaN,NaN,0,3,2010,WD,130500
1,1,544,531379050,60,RL,43.0,11492,Pave,None,IR1,...,0,0,NaN,NaN,NaN,0,4,2009,WD,220000
2,2,153,535304180,20,RL,68.0,7922,Pave,None,Reg,...,0,0,NaN,NaN,NaN,0,1,2010,WD,109000
3,3,318,916386060,60,RL,73.0,9802,Pave,None,Reg,...,0,0,NaN,NaN,NaN,0,4,2010,WD,174000
4,4,255,906425045,50,RL,82.0,14235,Pave,None,IR1,...,0,0,NaN,NaN,NaN,0,3,2010,WD,138500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2045,2046,1587,921126030,20,RL,79.0,11449,Pave,None,IR1,...,0,0,NaN,NaN,NaN,0,1,2008,WD,298751
2046,2047,785,905377130,30,RL,NaN,12342,Pave,None,IR1,...,0,0,NaN,NaN,NaN,0,3,2009,WD,82500
2047,2048,916,909253010,50,RL,57.0,7558,Pave,None,Reg,...,0,0,NaN,NaN,NaN,0,3,2009,WD,177000
2048,2049,639,535179160,20,RL,80.0,10400,Pave,None,Reg,...,0,0,NaN,NaN,NaN,0,11,2009,WD,144000


In [46]:
garage_index = df[(df['garage_yr_blt'].isnull()) & (df['garage_type'].notnull())].index

df.iloc[garage_index]['garage_type'].map({'garage_type': 'No Basement'})

df.iloc[garage_index]['garage_type']

1713    Attchd
Name: garage_type, dtype: object

In [47]:
garage_index.astype(int)

Int64Index([1712], dtype='int64')

In [48]:
df.at[1712, 'garage_type'] = 'No Garage'

In [49]:
df[(df['garage_yr_blt'].isnull()) & (df['garage_type'].notnull())][garage_list]

,garage_type,garage_yr_blt,garage_finish,garage_cars,garage_area,garage_qual,garage_cond
1712,No Garage,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:
df.iloc[garage_index][garage_list]

,garage_type,garage_yr_blt,garage_finish,garage_cars,garage_area,garage_qual,garage_cond
1713,Attchd,1952.0,RFn,1.0,280.0,TA,TA


In [51]:
df[(df['garage_yr_blt'].isnull()) & (df['garage_type'].notnull())][garage_list]

,garage_type,garage_yr_blt,garage_finish,garage_cars,garage_area,garage_qual,garage_cond
1712,No Garage,NaN,NaN,NaN,NaN,NaN,NaN


In [52]:
df['garage_type'] = df['garage_type'].fillna('No Garage')
   
df['garage_type'].isnull().sum()

0

The dictionary spells out the below "no_garage_list" columns as NA = No Garage. This is the first place I copied the df because I wanted a backup in case I made a mistake filling the NAs in multiple columns in one block, as it's the first time I've done it that way. When I succeeded, I moved the copy_df code to afterwards to save it before my next experiment.

In [53]:
no_garage_list = ['garage_finish', 'garage_qual', 'garage_cond']

df[no_garage_list] = df[no_garage_list].fillna('No Garage')

In [54]:
copy_df = df.copy()

In [55]:
df[df['garage_cars'].isnull()][garage_list]

,garage_type,garage_yr_blt,garage_finish,garage_cars,garage_area,garage_qual,garage_cond
1712,No Garage,NaN,No Garage,NaN,NaN,No Garage,No Garage


In [56]:
no_garage_list2 = ['garage_yr_blt', 'garage_cars', 'garage_area']

df.at[1712, no_garage_list2] = 'No Garage'

In [57]:
df[df['garage_cars'].isnull()][garage_list]

,garage_type,garage_yr_blt,garage_finish,garage_cars,garage_area,garage_qual,garage_cond


I've got to figure out what's going on with 'garage_yr_blt' because it seems possible some of these NAs may be for things other than "No Garage".

In [58]:
df[df['garage_yr_blt'].isnull()].head()[garage_list]

,garage_type,garage_yr_blt,garage_finish,garage_cars,garage_area,garage_qual,garage_cond
28,No Garage,NaN,No Garage,0.0,0.0,No Garage,No Garage
53,No Garage,NaN,No Garage,0.0,0.0,No Garage,No Garage
65,No Garage,NaN,No Garage,0.0,0.0,No Garage,No Garage
79,No Garage,NaN,No Garage,0.0,0.0,No Garage,No Garage
101,No Garage,NaN,No Garage,0.0,0.0,No Garage,No Garage


In [59]:
df[(df['garage_yr_blt'].isnull()) & (df['garage_finish'] != 'No Garage')]

,id,pid,ms_subclass,ms_zoning,lot_frontage,lot_area,street,alley,lot_shape,land_contour,...,screen_porch,pool_area,pool_qc,fence,misc_feature,misc_val,mo_sold,yr_sold,sale_type,saleprice


It appears all the garage_yr_built = NaN rows correspond to No Garage.

In [60]:
df['garage_yr_blt'] = df['garage_yr_blt'].fillna('No Garage')

In [61]:
df[df['garage_yr_blt'].isnull()].head()[garage_list]

,garage_type,garage_yr_blt,garage_finish,garage_cars,garage_area,garage_qual,garage_cond


I'm revisiting the null list to see what's left. I came back up to this one a few times, rather than rewriting it below, for additional sanity-check and to keep track of what's left.

In [62]:
df.columns[df.isna().any()].tolist()

['lot_frontage',
 'mas_vnr_type',
 'mas_vnr_area',
 'fireplace_qu',
 'pool_qc',
 'fence',
 'misc_feature']

#### Lot Frontage

In [63]:
df['lot_frontage'].value_counts(dropna = False)

NaN      330
60.0     179
70.0      96
80.0      94
50.0      90
        ... 
119.0      1
122.0      1
22.0       1
155.0      1
135.0      1
Name: lot_frontage, Length: 119, dtype: int64

In [64]:
df['lot_frontage'].isna().sum()/len(df)

0.16097560975609757

16% of the data seems like a lot to drop.

In [65]:
df[df['lot_frontage'] == 0.0]

,id,pid,ms_subclass,ms_zoning,lot_frontage,lot_area,street,alley,lot_shape,land_contour,...,screen_porch,pool_area,pool_qc,fence,misc_feature,misc_val,mo_sold,yr_sold,sale_type,saleprice


In [66]:
df[df['lot_frontage'] < 30.0]

,id,pid,ms_subclass,ms_zoning,lot_frontage,lot_area,street,alley,lot_shape,land_contour,...,screen_porch,pool_area,pool_qc,fence,misc_feature,misc_val,mo_sold,yr_sold,sale_type,saleprice
10,1044,527451290,160,RM,21.0,1680,Pave,None,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,7,2008,COD,85400
13,1177,533236070,160,FV,24.0,2645,Pave,Pave,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,12,2008,ConLD,200000
53,330,923226250,160,RM,21.0,1476,Pave,None,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,3,2010,WD,76000
56,98,533212020,160,FV,24.0,2544,Pave,Pave,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,2,2010,WD,149500
73,2832,908188140,160,RM,24.0,2522,Pave,None,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,4,2006,WD,137500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1987,2377,527453070,120,RL,24.0,2280,Pave,None,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,4,2006,WD,137500
2004,1843,533213130,160,FV,24.0,2280,Pave,Pave,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,9,2007,WD,176500
2006,1041,527450250,160,RM,21.0,1680,Pave,None,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,11,2008,WD,100000
2007,980,923228230,160,RM,21.0,2217,Pave,None,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,8,2009,WD,88000


It appears possible that NaN means no frontage because there are no 0.0 lots. In fact, there's nothing less than 21.0 feet of frontage. In reading a little about "flag lots" [here](https://www.city-data.com/forum/real-estate/1735402-value-land-without-road-frontage.html), it seems plausible that's what these are, and while 21 feet seems a little small from that reading, it seems possible these smaller lot frontages represent lots that include a path to the road in the property (as opposed to an easement across someone else's property). For now, I'm going to cast those as 0.0.

In [67]:
df['lot_frontage'] = df['lot_frontage'].fillna(0.0)

df['lot_frontage'].isna().sum()

0

### Fireplace Quality

The dictionary spells out that NA == no fireplace.

In [68]:
df['fireplace_qu'] = df['fireplace_qu'].fillna('No Fireplace')

df['fireplace_qu'].isna().sum()

0

### Pool Quality

The dictionary spells out that NA == no pool.

In [69]:
df['pool_qc'] = df['pool_qc'].fillna('No Pool')

df['fireplace_qu'].isna().sum()

0

### Fence

The dictionary spells out that NA == no fence.

In [70]:
df['fence'] = df['fence'].fillna('No Fence')

df['fence'].isna().sum()

0

### Misc Features

The dictionary spells out that NA == no additional features

In [71]:
df['misc_feature'] = df['misc_feature'].fillna('No Addl Features')
df['misc_feature'].isna().sum()

0

### Masonry Veneer Type

I examined these initially, and wasn't sure about how to handle them. I leaned toward dropping the 22 NaN since it's clear None is another category and hard to know how these NaN should be interpreted. With only 22 instances, I decided to drop these data.

In [72]:
df['mas_vnr_type'].value_counts(dropna = False)

None       1218
BrkFace     629
Stone       168
NaN          22
BrkCmn       13
Name: mas_vnr_type, dtype: int64

In [73]:
df[df['mas_vnr_type'].isna()]

,id,pid,ms_subclass,ms_zoning,lot_frontage,lot_area,street,alley,lot_shape,land_contour,...,screen_porch,pool_area,pool_qc,fence,misc_feature,misc_val,mo_sold,yr_sold,sale_type,saleprice
22,2393,528142010,60,RL,103.0,12867,Pave,None,IR1,Lvl,...,0,0,No Pool,No Fence,No Addl Features,0,7,2006,New,344133
41,2383,528110050,20,RL,107.0,13891,Pave,None,Reg,Lvl,...,0,0,No Pool,No Fence,No Addl Features,0,9,2006,New,465000
86,539,531371050,20,RL,67.0,10083,Pave,None,Reg,Lvl,...,0,0,No Pool,No Fence,No Addl Features,0,8,2009,WD,184900
212,518,528458020,20,FV,90.0,7993,Pave,None,IR1,Lvl,...,0,0,No Pool,No Fence,No Addl Features,0,10,2009,New,225000
276,2824,908130020,20,RL,75.0,8050,Pave,None,Reg,Lvl,...,0,0,No Pool,No Fence,No Addl Features,0,4,2006,WD,117250
338,1800,528458150,60,FV,112.0,12217,Pave,None,IR1,Lvl,...,0,0,No Pool,No Fence,No Addl Features,0,12,2007,New,310013
431,1455,907251090,60,RL,75.0,9473,Pave,None,Reg,Lvl,...,0,0,No Pool,No Fence,No Addl Features,0,3,2008,WD,237000
451,1120,528439010,20,RL,87.0,10037,Pave,None,Reg,Lvl,...,0,0,No Pool,No Fence,No Addl Features,0,8,2008,WD,247000
591,1841,533208040,120,FV,35.0,4274,Pave,Pave,IR1,Lvl,...,0,0,No Pool,No Fence,No Addl Features,0,11,2007,New,199900
844,1840,533208030,120,FV,30.0,5330,Pave,Pave,IR2,Lvl,...,0,0,No Pool,No Fence,No Addl Features,0,7,2007,New,207500


### Masonry Veneer Area

In [74]:
df['mas_vnr_area'].value_counts(dropna = False)

0.0      1216
NaN        22
120.0      11
176.0      10
200.0      10
         ... 
142.0       1
215.0       1
235.0       1
233.0       1
426.0       1
Name: mas_vnr_area, Length: 374, dtype: int64

In [75]:
len(df[(df['mas_vnr_type'].isna()) & (df['mas_vnr_area'].isna())])

22

The nulls from Masonry Veneer Type and Area are the same ones.

In [76]:
copy_df = df

I reviewed the .dropna documentation to understand how to target this to the column I wanted (which had the effect of taking care of the other column, since it was the same rows with the relevant NA)

In [77]:
df.dropna(subset = 'mas_vnr_type', inplace = True)

In [78]:
len(df[(df['mas_vnr_type'].isna()) & (df['mas_vnr_area'].isna())])

0

-----

# Exporting Dataframe to CSV

In [80]:
df.to_csv('datasets/draft1_cleaned_train.csv', index = False)

-----

# Examining Data for Outliers
I did additional cleaning on the data after I'd made my first round of predictions. Ultimately, however, I found that the data yielded weaker predictions, so I decided not to proceed with them. I've left the work here, though, to show the thought process.

In [108]:
df.describe()

,id,pid,ms_subclass,lot_frontage,lot_area,overall_qual,overall_cond,year_built,year_remod/add,mas_vnr_area,bsmtfin_sf_1,bsmtfin_sf_2,bsmt_unf_sf,total_bsmt_sf,1st_flr_sf,2nd_flr_sf,low_qual_fin_sf,gr_liv_area,bsmt_full_bath,bsmt_half_bath,full_bath,half_bath,bedroom_abvgr,kitchen_abvgr,totrms_abvgrd,fireplaces,wood_deck_sf,open_porch_sf,enclosed_porch,3ssn_porch,screen_porch,pool_area,misc_val,mo_sold,yr_sold,saleprice
count,2028.000000,2.028000e+03,2028.000000,2028.000000,2028.000000,2028.000000,2028.000000,2028.000000,2028.000000,2028.000000,2028.000000,2028.000000,2028.000000,2028.000000,2028.000000,2028.000000,2028.000000,2028.000000,2028.000000,2028.000000,2028.000000,2028.000000,2028.000000,2028.000000,2028.000000,2028.000000,2028.000000,2028.000000,2028.000000,2028.000000,2028.000000,2028.000000,2028.00000,2028.000000,2028.000000,2028.000000
mean,1473.767751,7.149286e+08,57.122781,57.966963,10053.849606,6.098126,5.569034,1971.357495,1983.967456,99.599112,441.290434,48.096647,565.328402,1054.715483,1163.274655,328.687377,5.575444,1497.537475,0.426529,0.064103,1.571992,0.367850,2.844181,1.042899,6.434911,0.589250,94.146943,47.139546,22.630671,2.620809,16.698718,2.425049,52.15927,6.206114,2007.778107,180839.170118
std,845.058814,1.887115e+08,42.930897,33.069068,6755.099389,1.424895,1.107746,30.147866,21.038807,174.951931,460.530353,165.456384,443.937563,447.488378,395.715616,425.317335,51.354392,501.541089,0.522816,0.252922,0.547898,0.499424,0.827727,0.209856,1.563396,0.638683,128.888850,66.445075,59.880980,25.370830,57.671676,37.995453,576.61144,2.741255,1.314063,79109.425782
min,1.000000,5.263011e+08,20.000000,0.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,0.000000,0.000000,0.000000,334.000000,0.000000,0.000000,334.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,1.000000,2006.000000,12789.000000
25%,752.750000,5.284771e+08,20.000000,44.000000,7500.000000,5.000000,5.000000,1953.000000,1964.000000,0.000000,0.000000,0.000000,219.750000,791.750000,879.750000,0.000000,0.000000,1126.000000,0.000000,0.000000,1.000000,0.000000,2.000000,1.000000,5.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,4.000000,2007.000000,129500.000000
50%,1483.500000,5.354551e+08,50.000000,63.000000,9400.000000,6.000000,5.000000,1973.000000,1993.000000,0.000000,368.000000,0.000000,473.500000,993.000000,1092.000000,0.000000,0.000000,1442.000000,0.000000,0.000000,2.000000,0.000000,3.000000,1.000000,6.000000,1.000000,0.000000,26.000000,0.000000,0.000000,0.000000,0.000000,0.00000,6.000000,2008.000000,161000.000000
75%,2197.500000,9.071804e+08,70.000000,78.000000,11500.000000,7.000000,6.000000,2000.000000,2004.000000,160.250000,732.250000,0.000000,809.250000,1314.000000,1402.000000,689.250000,0.000000,1728.000000,1.000000,0.000000,2.000000,1.000000,3.000000,1.000000,7.000000,1.000000,168.000000,70.000000,0.000000,0.000000,0.000000,0.000000,0.00000,8.000000,2009.000000,213310.000000
max,2930.000000,9.241520e+08,190.000000,313.000000,159000.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,1474.000000,2336.000000,6110.000000,5095.000000,1862.000000,1064.000000,5642.000000,3.000000,2.000000,4.000000,2.000000,8.000000,3.000000,15.000000,4.000000,1424.000000,547.000000,432.000000,508.000000,490.000000,800.000000,17000.00000,12.000000,2010.000000,611657.000000


Outliers I need to examime:
* revisit the 0.0 frontage. Is that an error? should it be removed?
* lot area - max lot ~15 x 75th percentile. 159,000 sq ft is 3.6 acres. That doesn't seem unreasonable, but could it be skewing the data?
* mas_vnr_area 1600 sq ft max is 10 times the 75th percentile.
* wood deck sq ft 1424 sf max is ~9x the 75th percentile
* open_porch max of 547 sf is ~8 times 75th percentile
* enclosed porch, other two porches, pool area, misc value, all more than 75% 0.0.
* saleprice - just check how far out the outliers are, both high and low. Lowest one here is about $12k, which seems off, given 25th percentile is $125k

From below:
* 3 kitchens above ground seems like a potential outlier, too.

Otherwise, it seems like there may just be a few huge houses. Or at least one. I'm going to look into that.

In [94]:
col_dtypes = pd.DataFrame(df.dtypes)
col_dtypes.head()

,0
id,int64
pid,int64
ms_subclass,int64
ms_zoning,object
lot_frontage,float64


In [95]:
col_dtypes[(col_dtypes[0] == 'int64')]

,0
id,int64
pid,int64
ms_subclass,int64
lot_area,int64
overall_qual,int64
overall_cond,int64
year_built,int64
year_remod/add,int64
1st_flr_sf,int64
2nd_flr_sf,int64


In [93]:
col_dtypes[(col_dtypes[0] == 'float64')]

,0
lot_frontage,float64
mas_vnr_area,float64
bsmtfin_sf_1,float64
bsmtfin_sf_2,float64
bsmt_unf_sf,float64
total_bsmt_sf,float64
bsmt_full_bath,float64
bsmt_half_bath,float64


The below two describes are to see the things that didn't pop up above.

In [96]:
df[['bsmtfin_sf_1',
'bsmtfin_sf_2',
'bsmt_unf_sf',
'total_bsmt_sf',
'bsmt_full_bath',
'bsmt_half_bath']].describe()

,bsmtfin_sf_1,bsmtfin_sf_2,bsmt_unf_sf,total_bsmt_sf,bsmt_full_bath,bsmt_half_bath
count,2028.000000,2028.000000,2028.000000,2028.000000,2028.000000,2028.000000
mean,441.290434,48.096647,565.328402,1054.715483,0.426529,0.064103
std,460.530353,165.456384,443.937563,447.488378,0.522816,0.252922
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,219.750000,791.750000,0.000000,0.000000
50%,368.000000,0.000000,473.500000,993.000000,0.000000,0.000000
75%,732.250000,0.000000,809.250000,1314.000000,1.000000,0.000000
max,5644.000000,1474.000000,2336.000000,6110.000000,3.000000,2.000000


In [97]:
df[['1st_flr_sf',
'2nd_flr_sf',
'low_qual_fin_sf',
'gr_liv_area',
'full_bath',
'half_bath',
'bedroom_abvgr',
'kitchen_abvgr',
'totrms_abvgrd',
'fireplaces']].describe()

,1st_flr_sf,2nd_flr_sf,low_qual_fin_sf,gr_liv_area,full_bath,half_bath,bedroom_abvgr,kitchen_abvgr,totrms_abvgrd,fireplaces
count,2028.000000,2028.000000,2028.000000,2028.000000,2028.000000,2028.000000,2028.000000,2028.000000,2028.000000,2028.000000
mean,1163.274655,328.687377,5.575444,1497.537475,1.571992,0.367850,2.844181,1.042899,6.434911,0.589250
std,395.715616,425.317335,51.354392,501.541089,0.547898,0.499424,0.827727,0.209856,1.563396,0.638683
min,334.000000,0.000000,0.000000,334.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000
25%,879.750000,0.000000,0.000000,1126.000000,1.000000,0.000000,2.000000,1.000000,5.000000,0.000000
50%,1092.000000,0.000000,0.000000,1442.000000,2.000000,0.000000,3.000000,1.000000,6.000000,1.000000
75%,1402.000000,689.250000,0.000000,1728.000000,2.000000,1.000000,3.000000,1.000000,7.000000,1.000000
max,5095.000000,1862.000000,1064.000000,5642.000000,4.000000,2.000000,8.000000,3.000000,15.000000,4.000000


[This site](https://datascienceparichay.com/article/show-all-columns-of-pandas-dataframe-in-jupyter-notebook/) showed me how to display all columns.

In [100]:
pd.set_option('display.max_columns', None)

In [107]:
df[df['saleprice'] > 500_000]

,id,pid,ms_subclass,ms_zoning,lot_frontage,lot_area,street,alley,lot_shape,land_contour,utilities,lot_config,land_slope,neighborhood,condition_1,condition_2,bldg_type,house_style,overall_qual,overall_cond,year_built,year_remod/add,roof_style,roof_matl,exterior_1st,exterior_2nd,mas_vnr_type,mas_vnr_area,exter_qual,exter_cond,foundation,bsmt_qual,bsmt_cond,bsmt_exposure,bsmtfin_type_1,bsmtfin_sf_1,bsmtfin_type_2,bsmtfin_sf_2,bsmt_unf_sf,total_bsmt_sf,heating,heating_qc,central_air,electrical,1st_flr_sf,2nd_flr_sf,low_qual_fin_sf,gr_liv_area,bsmt_full_bath,bsmt_half_bath,full_bath,half_bath,bedroom_abvgr,kitchen_abvgr,kitchen_qual,totrms_abvgrd,functional,fireplaces,fireplace_qu,garage_type,garage_yr_blt,garage_finish,garage_cars,garage_area,garage_qual,garage_cond,paved_drive,wood_deck_sf,open_porch_sf,enclosed_porch,3ssn_porch,screen_porch,pool_area,pool_qc,fence,misc_feature,misc_val,mo_sold,yr_sold,sale_type,saleprice
81,367,527214050,20,RL,63.0,17423,Pave,None,IR1,Lvl,AllPub,CulDSac,Gtl,StoneBr,Norm,Norm,1Fam,1Story,9,5,2008,2009,Hip,CompShg,VinylSd,VinylSd,Stone,748.0,Ex,TA,PConc,Ex,TA,No,GLQ,1904.0,Unf,0.0,312.0,2216.0,GasA,Ex,Y,SBrkr,2234,0,0,2234,1.0,0.0,2,0,1,1,Ex,9,Typ,1,Gd,Attchd,2009.0,Fin,3.0,1166.0,TA,TA,Y,0,60,0,0,0,0,No Pool,No Fence,No Addl Features,0,7,2009,New,501837
138,2331,527210040,60,RL,60.0,18062,Pave,None,IR1,HLS,AllPub,CulDSac,Gtl,StoneBr,Norm,Norm,1Fam,2Story,10,5,2006,2006,Hip,CompShg,CemntBd,CmentBd,BrkFace,662.0,Ex,TA,PConc,Ex,TA,Gd,Unf,0.0,Unf,0.0,1528.0,1528.0,GasA,Ex,Y,SBrkr,1528,1862,0,3390,0.0,0.0,3,1,5,1,Ex,10,Typ,1,Ex,BuiltIn,2006.0,Fin,3.0,758.0,TA,TA,Y,204,34,0,0,0,0,No Pool,No Fence,No Addl Features,0,9,2006,New,545224
151,2333,527212030,60,RL,85.0,16056,Pave,None,IR1,Lvl,AllPub,Inside,Gtl,StoneBr,Norm,Norm,1Fam,2Story,9,5,2005,2006,Hip,CompShg,CemntBd,CmentBd,Stone,208.0,Gd,TA,PConc,Ex,TA,Av,GLQ,240.0,Unf,0.0,1752.0,1992.0,GasA,Ex,Y,SBrkr,1992,876,0,2868,0.0,0.0,3,1,4,1,Ex,11,Typ,1,Gd,BuiltIn,2005.0,Fin,3.0,716.0,TA,TA,Y,214,108,0,0,0,0,No Pool,No Fence,No Addl Features,0,7,2006,New,556581
623,457,528176030,20,RL,100.0,14836,Pave,None,IR1,HLS,AllPub,Inside,Mod,NridgHt,Norm,Norm,1Fam,1Story,10,5,2004,2005,Hip,CompShg,CemntBd,CmentBd,Stone,730.0,Ex,TA,PConc,Ex,TA,Gd,GLQ,2146.0,Unf,0.0,346.0,2492.0,GasA,Ex,Y,SBrkr,2492,0,0,2492,1.0,0.0,2,1,2,1,Ex,8,Typ,1,Ex,Attchd,2004.0,Fin,3.0,949.0,TA,TA,Y,226,235,0,0,0,0,No Pool,No Fence,No Addl Features,0,2,2009,WD,552000
800,1702,528118050,20,RL,59.0,17169,Pave,None,IR2,Lvl,AllPub,CulDSac,Gtl,NridgHt,Norm,Norm,1Fam,1Story,10,5,2007,2007,Hip,CompShg,CemntBd,CmentBd,BrkFace,970.0,Ex,TA,PConc,Ex,TA,Av,GLQ,1684.0,Unf,0.0,636.0,2320.0,GasA,Ex,Y,SBrkr,2290,0,0,2290,2.0,0.0,2,1,2,1,Ex,7,Typ,1,Gd,Attchd,2007.0,Fin,3.0,1174.0,TA,TA,Y,192,30,0,0,0,0,No Pool,No Fence,No Addl Features,0,8,2007,New,500067
823,16,527216070,60,RL,47.0,53504,Pave,None,IR2,HLS,AllPub,CulDSac,Mod,StoneBr,Norm,Norm,1Fam,2Story,8,5,2003,2003,Hip,CompShg,CemntBd,Wd Shng,BrkFace,603.0,Ex,TA,PConc,Gd,TA,Gd,ALQ,1416.0,Unf,0.0,234.0,1650.0,GasA,Ex,Y,SBrkr,1690,1589,0,3279,1.0,0.0,3,1,4,1,Ex,12,Mod,1,Gd,BuiltIn,2003.0,Fin,3.0,841.0,TA,TA,Y,503,36,0,0,210,0,No Pool,No Fence,No Addl Features,0,6,2010,WD,538000
1164,424,528106020,20,RL,105.0,15431,Pave,None,Reg,Lvl,AllPub,Inside,Gtl,NridgHt,Norm,Norm,1Fam,1Story,10,5,2008,2008,Hip,CompShg,VinylSd,VinylSd,Stone,200.0,Ex,TA,PConc,Ex,TA,Gd,GLQ,1767.0,ALQ,539.0,788.0,3094.0,GasA,Ex,Y,SBrkr,2402,0,0,2402,1.0,0.0,2,0,2,1,Ex,10,Typ,2,Gd,Attchd,2008.0,Fin,3.0,672.0,TA,TA,Y,0,72,0,0,170,0,No Pool,No Fence,No Addl Features,0,4,2009,WD,555000
1592,2335,527214060,60,RL,82.0,16052,Pave,None,IR1,Lvl,AllPub,CulDSac,Gtl,StoneBr,Norm,Norm,1Fam,2Story,10,5,2006,2006,Hip,CompShg,VinylSd,VinylSd,Stone,734.0,Ex,TA,PConc,Ex,TA,No,GLQ,1206.0,Unf,0.0,644.0,1850.0,GasA,Ex,Y,SBrkr,1850,848,0,2698,1.0,0.0,2,1,4,1,Ex,11,Typ,1,Gd,Attchd,2006.0,RFn,3.0,736.0,TA,TA,Y,250,0,0,0,0,0,No Pool,No Fence,No Addl Features,0,7,2006,New,535000
1671,45,528150070,20,RL,100.0,12919,Pave,No

Nothing about the high sale prices seems unusual to me.

In [112]:
df[df['lot_area'] > 50_000][['lot_area', 'saleprice', 'gr_liv_area']]

,lot_area,saleprice,gr_liv_area
471,159000,277000,2144
694,115149,302000,1824
745,57200,160000,1687
823,53504,538000,3279
960,63887,160000,5642
1052,53227,256000,1663
1571,50271,385000,1842
1726,50102,250764,1650
1843,53107,240000,1953
1854,70761,280000,1533


The two lots over 100,000 sf are so far out there and their prices are so similar to smaller lots, that I think they're in error. I'm going to eliminate them.

In [113]:
df2 = df.copy()

In [114]:
df2.drop(index = df[df['lot_area'] > 100_000].index, inplace = True)

In [121]:
df2[df2['lot_area'] > 100_000][['lot_area', 'saleprice', 'gr_liv_area']]

,lot_area,saleprice,gr_liv_area


In [122]:
df2[df2['mas_vnr_area']>1000][['mas_vnr_area', 'saleprice', 'year_built']]

,mas_vnr_area,saleprice,year_built
125,1050.0,150000,1977
378,1115.0,244000,1973
489,1110.0,421250,2001
1151,1129.0,176000,1999
1170,1031.0,438780,2006
1227,1095.0,500000,2003
1416,1047.0,159950,1966
1885,1224.0,183850,2008


The 1600 SF masonry veneer area seems so far out there, and the price is so much lower than others, I'm going to eliminate it.

In [123]:
df2.drop(index = df2[df2['mas_vnr_area'] == 1600].index, inplace = True)

In [124]:
df2[df2['mas_vnr_area'] > 1300][['mas_vnr_area', 'saleprice', 'year_built']]

,mas_vnr_area,saleprice,year_built


In [126]:
df2[df2['wood_deck_sf']>800]

,id,pid,ms_subclass,ms_zoning,lot_frontage,lot_area,street,alley,lot_shape,land_contour,utilities,lot_config,land_slope,neighborhood,condition_1,condition_2,bldg_type,house_style,overall_qual,overall_cond,year_built,year_remod/add,roof_style,roof_matl,exterior_1st,exterior_2nd,mas_vnr_type,mas_vnr_area,exter_qual,exter_cond,foundation,bsmt_qual,bsmt_cond,bsmt_exposure,bsmtfin_type_1,bsmtfin_sf_1,bsmtfin_type_2,bsmtfin_sf_2,bsmt_unf_sf,total_bsmt_sf,heating,heating_qc,central_air,electrical,1st_flr_sf,2nd_flr_sf,low_qual_fin_sf,gr_liv_area,bsmt_full_bath,bsmt_half_bath,full_bath,half_bath,bedroom_abvgr,kitchen_abvgr,kitchen_qual,totrms_abvgrd,functional,fireplaces,fireplace_qu,garage_type,garage_yr_blt,garage_finish,garage_cars,garage_area,garage_qual,garage_cond,paved_drive,wood_deck_sf,open_porch_sf,enclosed_porch,3ssn_porch,screen_porch,pool_area,pool_qc,fence,misc_feature,misc_val,mo_sold,yr_sold,sale_type,saleprice
966,2294,923229100,80,RL,0.0,15957,Pave,None,IR1,Low,AllPub,Corner,Mod,Mitchel,Norm,Norm,1Fam,SLvl,6,6,1977,1977,Gable,CompShg,HdBoard,Plywood,None,0.0,TA,TA,PConc,Gd,TA,Gd,GLQ,1148.0,Unf,0.0,96.0,1244.0,GasA,TA,Y,SBrkr,1356,0,0,1356,2.0,0.0,2,0,3,1,TA,6,Typ,1,TA,Attchd,1977.0,Fin,2.0,528.0,TA,TA,Y,1424,0,0,0,0,0,No Pool,MnPrv,No Addl Features,0,9,2007,WD,188000
1571,2523,533350050,20,RL,68.0,50271,Pave,None,IR1,Low,AllPub,Inside,Gtl,Veenker,Norm,Norm,1Fam,1Story,9,5,1981,1987,Gable,WdShngl,WdShing,Wd Shng,None,0.0,Gd,TA,CBlock,Ex,TA,Gd,GLQ,1810.0,Unf,0.0,32.0,1842.0,GasA,Gd,Y,SBrkr,1842,0,0,1842,2.0,0.0,0,1,0,1,Gd,5,Typ,1,Gd,Attchd,1981.0,Fin,3.0,894.0,TA,TA,Y,857,72,0,0,0,0,No Pool,No Fence,No Addl Features,0,11,2006,WD,385000
1692,2451,528360050,60,RL,114.0,17242,Pave,None,IR1,Lvl,AllPub,Inside,Gtl,NoRidge,Norm,Norm,1Fam,2Story,9,5,1993,1994,Hip,CompShg,MetalSd,MetalSd,BrkFace,738.0,Gd,Gd,PConc,Ex,TA,Gd,Rec,292.0,GLQ,1393.0,48.0,1733.0,GasA,Ex,Y,SBrkr,1933,1567,0,3500,1.0,0.0,3,1,4,1,Ex,11,Typ,1,TA,Attchd,1993.0,RFn,3.0,959.0,TA,TA,Y,870,86,0,0,210,0,No Pool,No Fence,No Addl Features,0,5,2006,WD,584500


The 1424 sf deck is so much larger than the others and the sale price so much lower, I'm going to eliminate it.

In [130]:
df2.drop(index = df2[df2['wood_deck_sf'] == 1424].index, inplace = True)

In [131]:
df2[df2['wood_deck_sf'] == 1400]

,id,pid,ms_subclass,ms_zoning,lot_frontage,lot_area,street,alley,lot_shape,land_contour,utilities,lot_config,land_slope,neighborhood,condition_1,condition_2,bldg_type,house_style,overall_qual,overall_cond,year_built,year_remod/add,roof_style,roof_matl,exterior_1st,exterior_2nd,mas_vnr_type,mas_vnr_area,exter_qual,exter_cond,foundation,bsmt_qual,bsmt_cond,bsmt_exposure,bsmtfin_type_1,bsmtfin_sf_1,bsmtfin_type_2,bsmtfin_sf_2,bsmt_unf_sf,total_bsmt_sf,heating,heating_qc,central_air,electrical,1st_flr_sf,2nd_flr_sf,low_qual_fin_sf,gr_liv_area,bsmt_full_bath,bsmt_half_bath,full_bath,half_bath,bedroom_abvgr,kitchen_abvgr,kitchen_qual,totrms_abvgrd,functional,fireplaces,fireplace_qu,garage_type,garage_yr_blt,garage_finish,garage_cars,garage_area,garage_qual,garage_cond,paved_drive,wood_deck_sf,open_porch_sf,enclosed_porch,3ssn_porch,screen_porch,pool_area,pool_qc,fence,misc_feature,misc_val,mo_sold,yr_sold,sale_type,saleprice


In [133]:
df2[df2['open_porch_sf'] > 500]

,id,pid,ms_subclass,ms_zoning,lot_frontage,lot_area,street,alley,lot_shape,land_contour,utilities,lot_config,land_slope,neighborhood,condition_1,condition_2,bldg_type,house_style,overall_qual,overall_cond,year_built,year_remod/add,roof_style,roof_matl,exterior_1st,exterior_2nd,mas_vnr_type,mas_vnr_area,exter_qual,exter_cond,foundation,bsmt_qual,bsmt_cond,bsmt_exposure,bsmtfin_type_1,bsmtfin_sf_1,bsmtfin_type_2,bsmtfin_sf_2,bsmt_unf_sf,total_bsmt_sf,heating,heating_qc,central_air,electrical,1st_flr_sf,2nd_flr_sf,low_qual_fin_sf,gr_liv_area,bsmt_full_bath,bsmt_half_bath,full_bath,half_bath,bedroom_abvgr,kitchen_abvgr,kitchen_qual,totrms_abvgrd,functional,fireplaces,fireplace_qu,garage_type,garage_yr_blt,garage_finish,garage_cars,garage_area,garage_qual,garage_cond,paved_drive,wood_deck_sf,open_porch_sf,enclosed_porch,3ssn_porch,screen_porch,pool_area,pool_qc,fence,misc_feature,misc_val,mo_sold,yr_sold,sale_type,saleprice
974,1289,902105020,50,RM,60.0,10440,Pave,Grvl,Reg,Lvl,AllPub,Corner,Gtl,OldTown,Norm,Norm,1Fam,1.5Fin,6,7,1920,1950,Gable,CompShg,BrkFace,Wd Sdng,None,0.0,Gd,Gd,BrkTil,Gd,TA,No,LwQ,493.0,Unf,0.0,1017.0,1510.0,GasW,Ex,Y,SBrkr,1584,1208,0,2792,0.0,0.0,2,0,5,1,TA,8,Mod,2,TA,Detchd,1920.0,Unf,2.0,520.0,Fa,TA,Y,0,547,0,0,480,0,No Pool,MnPrv,Shed,1150,6,2008,WD,256000
1141,1321,902401120,75,RM,75.0,13500,Pave,None,Reg,Lvl,AllPub,Inside,Gtl,OldTown,Artery,PosA,1Fam,2.5Unf,10,9,1893,2000,Gable,CompShg,Wd Sdng,Wd Sdng,None,0.0,Ex,Ex,BrkTil,TA,TA,No,Unf,0.0,Unf,0.0,1237.0,1237.0,GasA,Gd,Y,SBrkr,1521,1254,0,2775,0.0,0.0,3,1,3,1,Gd,9,Typ,1,Gd,Detchd,1988.0,Unf,2.0,880.0,Gd,TA,Y,105,502,0,0,0,0,No Pool,No Fence,No Addl Features,0,7,2008,WD,325000
1309,727,902477120,30,C (all),60.0,7879,Pave,None,Reg,Lvl,AllPub,Inside,Gtl,IDOTRR,Norm,Norm,1Fam,1Story,4,5,1920,1950,Gable,CompShg,Wd Sdng,Wd Sdng,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,495.0,Unf,0.0,225.0,720.0,GasA,TA,N,FuseA,720,0,0,720,0.0,0.0,1,0,2,1,TA,4,Typ,0,No Fireplace,No Garage,No Garage,No Garage,0.0,0.0,No Garage,No Garage,N,0,523,115,0,0,0,No Pool,GdWo,No Addl Features,0,11,2009,WD,34900


This actually doesn't seem that crazy except for the last one, coming in at $35k sale price. I have to check those low prices out.

In [143]:

df2[df2['saleprice']<35_000]

,id,pid,ms_subclass,ms_zoning,lot_frontage,lot_area,street,alley,lot_shape,land_contour,utilities,lot_config,land_slope,neighborhood,condition_1,condition_2,bldg_type,house_style,overall_qual,overall_cond,year_built,year_remod/add,roof_style,roof_matl,exterior_1st,exterior_2nd,mas_vnr_type,mas_vnr_area,exter_qual,exter_cond,foundation,bsmt_qual,bsmt_cond,bsmt_exposure,bsmtfin_type_1,bsmtfin_sf_1,bsmtfin_type_2,bsmtfin_sf_2,bsmt_unf_sf,total_bsmt_sf,heating,heating_qc,central_air,electrical,1st_flr_sf,2nd_flr_sf,low_qual_fin_sf,gr_liv_area,bsmt_full_bath,bsmt_half_bath,full_bath,half_bath,bedroom_abvgr,kitchen_abvgr,kitchen_qual,totrms_abvgrd,functional,fireplaces,fireplace_qu,garage_type,garage_yr_blt,garage_finish,garage_cars,garage_area,garage_qual,garage_cond,paved_drive,wood_deck_sf,open_porch_sf,enclosed_porch,3ssn_porch,screen_porch,pool_area,pool_qc,fence,misc_feature,misc_val,mo_sold,yr_sold,sale_type,saleprice
183,1554,910251050,20,A (agr),80.0,14584,Pave,None,Reg,Low,AllPub,Inside,Mod,IDOTRR,Norm,Norm,1Fam,1Story,1,5,1952,1952,Gable,CompShg,AsbShng,VinylSd,None,0.0,Fa,Po,Slab,No Basement,No Basement,No Basement,No Basement,0.0,No Basement,0.0,0.0,0.0,Wall,Po,N,FuseA,733,0,0,733,0.0,0.0,1,0,2,1,Fa,4,Sal,0,No Fireplace,Attchd,1952.0,Unf,2.0,487.0,Fa,Po,N,0,0,0,0,0,0,No Pool,No Fence,No Addl Features,0,2,2008,WD,13100
1309,727,902477120,30,C (all),60.0,7879,Pave,None,Reg,Lvl,AllPub,Inside,Gtl,IDOTRR,Norm,Norm,1Fam,1Story,4,5,1920,1950,Gable,CompShg,Wd Sdng,Wd Sdng,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,495.0,Unf,0.0,225.0,720.0,GasA,TA,N,FuseA,720,0,0,720,0.0,0.0,1,0,2,1,TA,4,Typ,0,No Fireplace,No Garage,No Garage,No Garage,0.0,0.0,No Garage,No Garage,N,0,523,115,0,0,0,No Pool,GdWo,No Addl Features,0,11,2009,WD,34900
1628,182,902207130,30,RM,68.0,9656,Pave,None,Reg,Lvl,AllPub,Inside,Gtl,OldTown,Norm,Norm,1Fam,1Story,2,2,1923,1970,Gable,CompShg,AsbShng,AsbShng,None,0.0,TA,Fa,BrkTil,Fa,Fa,No,Unf,0.0,Unf,0.0,678.0,678.0,GasA,TA,N,SBrkr,832,0,0,832,0.0,0.0,1,0,2,1,TA,5,Typ,1,Gd,Detchd,1928.0,Unf,2.0,780.0,Fa,Fa,N,0,0,0,0,0,0,No Pool,No Fence,No Addl Features,0,6,2010,WD,12789


The two properties that are less than $14,000 are so much less than the next closest that I'm going to eliminate them.

In [144]:
df2.drop(index = df2[df2['saleprice'] < 15_000].index, inplace = True)

In [145]:

df2[df2['saleprice']<35_000]

,id,pid,ms_subclass,ms_zoning,lot_frontage,lot_area,street,alley,lot_shape,land_contour,utilities,lot_config,land_slope,neighborhood,condition_1,condition_2,bldg_type,house_style,overall_qual,overall_cond,year_built,year_remod/add,roof_style,roof_matl,exterior_1st,exterior_2nd,mas_vnr_type,mas_vnr_area,exter_qual,exter_cond,foundation,bsmt_qual,bsmt_cond,bsmt_exposure,bsmtfin_type_1,bsmtfin_sf_1,bsmtfin_type_2,bsmtfin_sf_2,bsmt_unf_sf,total_bsmt_sf,heating,heating_qc,central_air,electrical,1st_flr_sf,2nd_flr_sf,low_qual_fin_sf,gr_liv_area,bsmt_full_bath,bsmt_half_bath,full_bath,half_bath,bedroom_abvgr,kitchen_abvgr,kitchen_qual,totrms_abvgrd,functional,fireplaces,fireplace_qu,garage_type,garage_yr_blt,garage_finish,garage_cars,garage_area,garage_qual,garage_cond,paved_drive,wood_deck_sf,open_porch_sf,enclosed_porch,3ssn_porch,screen_porch,pool_area,pool_qc,fence,misc_feature,misc_val,mo_sold,yr_sold,sale_type,saleprice
1309,727,902477120,30,C (all),60.0,7879,Pave,None,Reg,Lvl,AllPub,Inside,Gtl,IDOTRR,Norm,Norm,1Fam,1Story,4,5,1920,1950,Gable,CompShg,Wd Sdng,Wd Sdng,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,495.0,Unf,0.0,225.0,720.0,GasA,TA,N,FuseA,720,0,0,720,0.0,0.0,1,0,2,1,TA,4,Typ,0,No Fireplace,No Garage,No Garage,No Garage,0.0,0.0,No Garage,No Garage,N,0,523,115,0,0,0,No Pool,GdWo,No Addl Features,0,11,2009,WD,34900


In [146]:

df2[df2['saleprice']>500_000]

,id,pid,ms_subclass,ms_zoning,lot_frontage,lot_area,street,alley,lot_shape,land_contour,utilities,lot_config,land_slope,neighborhood,condition_1,condition_2,bldg_type,house_style,overall_qual,overall_cond,year_built,year_remod/add,roof_style,roof_matl,exterior_1st,exterior_2nd,mas_vnr_type,mas_vnr_area,exter_qual,exter_cond,foundation,bsmt_qual,bsmt_cond,bsmt_exposure,bsmtfin_type_1,bsmtfin_sf_1,bsmtfin_type_2,bsmtfin_sf_2,bsmt_unf_sf,total_bsmt_sf,heating,heating_qc,central_air,electrical,1st_flr_sf,2nd_flr_sf,low_qual_fin_sf,gr_liv_area,bsmt_full_bath,bsmt_half_bath,full_bath,half_bath,bedroom_abvgr,kitchen_abvgr,kitchen_qual,totrms_abvgrd,functional,fireplaces,fireplace_qu,garage_type,garage_yr_blt,garage_finish,garage_cars,garage_area,garage_qual,garage_cond,paved_drive,wood_deck_sf,open_porch_sf,enclosed_porch,3ssn_porch,screen_porch,pool_area,pool_qc,fence,misc_feature,misc_val,mo_sold,yr_sold,sale_type,saleprice
81,367,527214050,20,RL,63.0,17423,Pave,None,IR1,Lvl,AllPub,CulDSac,Gtl,StoneBr,Norm,Norm,1Fam,1Story,9,5,2008,2009,Hip,CompShg,VinylSd,VinylSd,Stone,748.0,Ex,TA,PConc,Ex,TA,No,GLQ,1904.0,Unf,0.0,312.0,2216.0,GasA,Ex,Y,SBrkr,2234,0,0,2234,1.0,0.0,2,0,1,1,Ex,9,Typ,1,Gd,Attchd,2009.0,Fin,3.0,1166.0,TA,TA,Y,0,60,0,0,0,0,No Pool,No Fence,No Addl Features,0,7,2009,New,501837
138,2331,527210040,60,RL,60.0,18062,Pave,None,IR1,HLS,AllPub,CulDSac,Gtl,StoneBr,Norm,Norm,1Fam,2Story,10,5,2006,2006,Hip,CompShg,CemntBd,CmentBd,BrkFace,662.0,Ex,TA,PConc,Ex,TA,Gd,Unf,0.0,Unf,0.0,1528.0,1528.0,GasA,Ex,Y,SBrkr,1528,1862,0,3390,0.0,0.0,3,1,5,1,Ex,10,Typ,1,Ex,BuiltIn,2006.0,Fin,3.0,758.0,TA,TA,Y,204,34,0,0,0,0,No Pool,No Fence,No Addl Features,0,9,2006,New,545224
151,2333,527212030,60,RL,85.0,16056,Pave,None,IR1,Lvl,AllPub,Inside,Gtl,StoneBr,Norm,Norm,1Fam,2Story,9,5,2005,2006,Hip,CompShg,CemntBd,CmentBd,Stone,208.0,Gd,TA,PConc,Ex,TA,Av,GLQ,240.0,Unf,0.0,1752.0,1992.0,GasA,Ex,Y,SBrkr,1992,876,0,2868,0.0,0.0,3,1,4,1,Ex,11,Typ,1,Gd,BuiltIn,2005.0,Fin,3.0,716.0,TA,TA,Y,214,108,0,0,0,0,No Pool,No Fence,No Addl Features,0,7,2006,New,556581
623,457,528176030,20,RL,100.0,14836,Pave,None,IR1,HLS,AllPub,Inside,Mod,NridgHt,Norm,Norm,1Fam,1Story,10,5,2004,2005,Hip,CompShg,CemntBd,CmentBd,Stone,730.0,Ex,TA,PConc,Ex,TA,Gd,GLQ,2146.0,Unf,0.0,346.0,2492.0,GasA,Ex,Y,SBrkr,2492,0,0,2492,1.0,0.0,2,1,2,1,Ex,8,Typ,1,Ex,Attchd,2004.0,Fin,3.0,949.0,TA,TA,Y,226,235,0,0,0,0,No Pool,No Fence,No Addl Features,0,2,2009,WD,552000
800,1702,528118050,20,RL,59.0,17169,Pave,None,IR2,Lvl,AllPub,CulDSac,Gtl,NridgHt,Norm,Norm,1Fam,1Story,10,5,2007,2007,Hip,CompShg,CemntBd,CmentBd,BrkFace,970.0,Ex,TA,PConc,Ex,TA,Av,GLQ,1684.0,Unf,0.0,636.0,2320.0,GasA,Ex,Y,SBrkr,2290,0,0,2290,2.0,0.0,2,1,2,1,Ex,7,Typ,1,Gd,Attchd,2007.0,Fin,3.0,1174.0,TA,TA,Y,192,30,0,0,0,0,No Pool,No Fence,No Addl Features,0,8,2007,New,500067
823,16,527216070,60,RL,47.0,53504,Pave,None,IR2,HLS,AllPub,CulDSac,Mod,StoneBr,Norm,Norm,1Fam,2Story,8,5,2003,2003,Hip,CompShg,CemntBd,Wd Shng,BrkFace,603.0,Ex,TA,PConc,Gd,TA,Gd,ALQ,1416.0,Unf,0.0,234.0,1650.0,GasA,Ex,Y,SBrkr,1690,1589,0,3279,1.0,0.0,3,1,4,1,Ex,12,Mod,1,Gd,BuiltIn,2003.0,Fin,3.0,841.0,TA,TA,Y,503,36,0,0,210,0,No Pool,No Fence,No Addl Features,0,6,2010,WD,538000
1164,424,528106020,20,RL,105.0,15431,Pave,None,Reg,Lvl,AllPub,Inside,Gtl,NridgHt,Norm,Norm,1Fam,1Story,10,5,2008,2008,Hip,CompShg,VinylSd,VinylSd,Stone,200.0,Ex,TA,PConc,Ex,TA,Gd,GLQ,1767.0,ALQ,539.0,788.0,3094.0,GasA,Ex,Y,SBrkr,2402,0,0,2402,1.0,0.0,2,0,2,1,Ex,10,Typ,2,Gd,Attchd,2008.0,Fin,3.0,672.0,TA,TA,Y,0,72,0,0,170,0,No Pool,No Fence,No Addl Features,0,4,2009,WD,555000
1592,2335,527214060,60,RL,82.0,16052,Pave,None,IR1,Lvl,AllPub,CulDSac,Gtl,StoneBr,Norm,Norm,1Fam,2Story,10,5,2006,2006,Hip,CompShg,VinylSd,VinylSd,Stone,734.0,Ex,TA,PConc,Ex,TA,No,GLQ,1206.0,Unf,0.0,644.0,1850.0,GasA,Ex,Y,SBrkr,1850,848,0,2698,1.0,0.0,2,1,4,1,Ex,11,Typ,1,Gd,Attchd,2006.0,RFn,3.0,736.0,TA,TA,Y,250,0,0,0,0,0,No Pool,No Fence,No Addl Features,0,7,2006,New,535000
1671,45,528150070,20,RL,100.0,12919,Pave,No

Nothing makes me think any of these higher priced ones are unreasonable.

enclosed porch, other two porches, pool area, misc value,

In [153]:
df2[df2['enclosed_porch']>300]

,id,pid,ms_subclass,ms_zoning,lot_frontage,lot_area,street,alley,lot_shape,land_contour,utilities,lot_config,land_slope,neighborhood,condition_1,condition_2,bldg_type,house_style,overall_qual,overall_cond,year_built,year_remod/add,roof_style,roof_matl,exterior_1st,exterior_2nd,mas_vnr_type,mas_vnr_area,exter_qual,exter_cond,foundation,bsmt_qual,bsmt_cond,bsmt_exposure,bsmtfin_type_1,bsmtfin_sf_1,bsmtfin_type_2,bsmtfin_sf_2,bsmt_unf_sf,total_bsmt_sf,heating,heating_qc,central_air,electrical,1st_flr_sf,2nd_flr_sf,low_qual_fin_sf,gr_liv_area,bsmt_full_bath,bsmt_half_bath,full_bath,half_bath,bedroom_abvgr,kitchen_abvgr,kitchen_qual,totrms_abvgrd,functional,fireplaces,fireplace_qu,garage_type,garage_yr_blt,garage_finish,garage_cars,garage_area,garage_qual,garage_cond,paved_drive,wood_deck_sf,open_porch_sf,enclosed_porch,3ssn_porch,screen_porch,pool_area,pool_qc,fence,misc_feature,misc_val,mo_sold,yr_sold,sale_type,saleprice
149,203,903426180,50,RM,57.0,8094,Pave,Grvl,Reg,Lvl,AllPub,Inside,Gtl,OldTown,Norm,Norm,1Fam,1.5Fin,4,5,1915,1950,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,Fa,CBlock,TA,TA,No,Unf,0.0,Unf,0.0,888.0,888.0,GasA,Ex,Y,SBrkr,888,1074,0,1962,0.0,0.0,1,1,4,1,TA,9,Typ,1,TA,Detchd,1915.0,Unf,2.0,572.0,TA,TA,Y,160,0,364,0,0,0,No Pool,GdPrv,No Addl Features,0,6,2010,WD,149500
241,2560,534475160,20,RL,89.0,10858,Pave,None,Reg,Lvl,AllPub,Corner,Gtl,NAmes,Feedr,Norm,1Fam,1Story,5,6,1952,1952,Gable,CompShg,Wd Sdng,Plywood,Stone,150.0,TA,Gd,CBlock,TA,TA,Mn,LwQ,40.0,Unf,0.0,1404.0,1444.0,GasA,Ex,Y,SBrkr,1624,0,0,1624,1.0,0.0,1,0,2,1,TA,6,Min1,1,Gd,Attchd,1952.0,RFn,1.0,240.0,TA,TA,Y,0,40,324,0,0,0,No Pool,MnPrv,No Addl Features,0,7,2006,WD,146500
507,989,924100060,60,RL,70.0,10457,Pave,None,IR1,Lvl,AllPub,Inside,Mod,Mitchel,Norm,Norm,1Fam,2Story,5,7,1969,1969,Gable,CompShg,VinylSd,VinylSd,BrkFace,178.0,Gd,Ex,CBlock,TA,TA,Gd,BLQ,496.0,LwQ,288.0,0.0,784.0,GasA,Ex,Y,SBrkr,784,848,0,1632,0.0,0.0,1,1,4,1,TA,7,Typ,1,TA,Attchd,1969.0,RFn,2.0,898.0,TA,TA,Y,0,173,368,0,0,0,No Pool,MnPrv,No Addl Features,0,4,2009,WD,173000
828,661,535381040,50,RL,60.0,10410,Pave,Grvl,Reg,Lvl,AllPub,Inside,Gtl,OldTown,Norm,Norm,1Fam,1.5Fin,4,5,1915,1950,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,BrkTil,TA,TA,No,Unf,0.0,Unf,0.0,1313.0,1313.0,GasA,TA,Y,SBrkr,1313,0,1064,2377,0.0,0.0,2,0,3,1,Gd,8,Min2,1,TA,Detchd,1954.0,Unf,2.0,528.0,TA,TA,Y,0,0,432,0,0,0,No Pool,No Fence,No Addl Features,0,6,2009,WD,142900
831,2663,902328100,75,RM,65.0,8850,Pave,None,IR1,Bnk,AllPub,Corner,Gtl,OldTown,Norm,Norm,1Fam,2.5Unf,7,6,1916,1950,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,BrkTil,TA,TA,No,Unf,0.0,Unf,0.0,815.0,815.0,GasA,Ex,Y,SBrkr,815,875,0,1690,0.0,0.0,1,0,3,1,TA,7,Typ,1,Gd,Detchd,1916.0,Unf,1.0,225.0,TA,TA,Y,0,0,330,0,0,0,No Pool,No Fence,No Addl Features,0,7,2006,ConLw,144000
1090,209,904100140,70,RL,0.0,24090,Pave,None,Reg,Lvl,AllPub,Inside,Gtl,ClearCr,Norm,Norm,1Fam,2Story,7,7,1940,1950,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,Gd,CBlock,TA,TA,Mn,Unf,0.0,Unf,0.0,1032.0,1032.0,GasA,Ex,Y,SBrkr,1207,1196,0,2403,0.0,0.0,2,0,4,1,TA,10,Typ,2,TA,Attchd,1940.0,Unf,1.0,349.0,TA,TA,Y,56,0,318,0,0,0,No Pool,No Fence,No Addl Features,0,6,2010,COD,244400
1782,1522,909250040,70,RL,51.0,9842,Pave,None,Reg,Lvl,AllPub,Inside,Gtl,SWISU,Feedr,Norm,1Fam,2Story,5,6,1921,1998,Gable,CompShg,MetalSd,Wd Sdng,None,0.0,TA,TA,BrkTil,TA,Fa,No,Unf,0.0,Unf,0.0,612.0,612.0,GasA,Ex,Y,SBrkr,990,1611,0,2601,0.0,0.0,3,1,4,1,TA,8,Typ,0,No Fireplace,BuiltIn,1998.0,RFn,2.0,621.0,TA,TA,Y,183,0,301,0,0,0,No Pool,No Fence,No Addl Features,0,5,2008,WD,189000


I don't see any reason to suspect that this isn't accurate.

In [154]:
df2[df2['3ssn_porch']>300]

,id,pid,ms_subclass,ms_zoning,lot_frontage,lot_area,street,alley,lot_shape,land_contour,utilities,lot_config,land_slope,neighborhood,condition_1,condition_2,bldg_type,house_style,overall_qual,overall_cond,year_built,year_remod/add,roof_style,roof_matl,exterior_1st,exterior_2nd,mas_vnr_type,mas_vnr_area,exter_qual,exter_cond,foundation,bsmt_qual,bsmt_cond,bsmt_exposure,bsmtfin_type_1,bsmtfin_sf_1,bsmtfin_type_2,bsmtfin_sf_2,bsmt_unf_sf,total_bsmt_sf,heating,heating_qc,central_air,electrical,1st_flr_sf,2nd_flr_sf,low_qual_fin_sf,gr_liv_area,bsmt_full_bath,bsmt_half_bath,full_bath,half_bath,bedroom_abvgr,kitchen_abvgr,kitchen_qual,totrms_abvgrd,functional,fireplaces,fireplace_qu,garage_type,garage_yr_blt,garage_finish,garage_cars,garage_area,garage_qual,garage_cond,paved_drive,wood_deck_sf,open_porch_sf,enclosed_porch,3ssn_porch,screen_porch,pool_area,pool_qc,fence,misc_feature,misc_val,mo_sold,yr_sold,sale_type,saleprice
892,1209,534400060,20,RL,100.0,10175,Pave,None,IR1,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,5,1964,1964,Gable,CompShg,HdBoard,Plywood,BrkFace,272.0,TA,TA,CBlock,TA,TA,No,BLQ,490.0,Unf,0.0,935.0,1425.0,GasA,Gd,Y,SBrkr,1425,0,0,1425,0.0,0.0,2,0,3,1,TA,7,Typ,1,Gd,Attchd,1964.0,RFn,2.0,576.0,TA,TA,Y,0,0,0,407,0,0,No Pool,No Fence,No Addl Features,0,7,2008,WD,180500
1219,365,527180100,20,RL,99.0,11851,Pave,None,Reg,Lvl,AllPub,Corner,Gtl,Gilbert,Norm,Norm,1Fam,1Story,7,5,1990,1990,Gable,CompShg,HdBoard,HdBoard,None,0.0,TA,TA,PConc,Gd,TA,No,Unf,0.0,Unf,0.0,1424.0,1424.0,GasA,Ex,Y,SBrkr,1442,0,0,1442,0.0,0.0,2,0,3,1,TA,5,Typ,0,No Fireplace,Attchd,1990.0,RFn,2.0,500.0,TA,TA,Y,0,34,0,508,0,0,No Pool,No Fence,No Addl Features,0,5,2009,WD,180500
1577,2592,535327210,20,RL,70.0,13300,Pave,None,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,1Fam,1Story,5,5,1956,2001,Hip,CompShg,Wd Sdng,VinylSd,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,494.0,Unf,0.0,521.0,1015.0,GasA,Gd,Y,SBrkr,1384,0,0,1384,1.0,0.0,1,0,2,1,TA,6,Min1,0,No Fireplace,Attchd,2001.0,Unf,2.0,896.0,TA,TA,Y,75,0,0,323,0,0,No Pool,No Fence,Shed,400,6,2006,WD,159000
2045,1051,528102030,20,RL,96.0,12444,Pave,None,Reg,Lvl,AllPub,FR2,Gtl,NridgHt,Norm,Norm,1Fam,1Story,8,5,2008,2008,Hip,CompShg,VinylSd,VinylSd,Stone,426.0,Ex,TA,PConc,Ex,TA,Av,GLQ,1336.0,Unf,0.0,596.0,1932.0,GasA,Ex,Y,SBrkr,1932,0,0,1932,1.0,0.0,2,0,2,1,Ex,7,Typ,1,Gd,Attchd,2008.0,Fin,3.0,774.0,TA,TA,Y,0,66,0,304,0,0,No Pool,No Fence,No Addl Features,0,11,2008,New,394617


Again nothing makes me suspect these aren't accurate.

In [155]:
df2[df2['screen_porch']>300]

,id,pid,ms_subclass,ms_zoning,lot_frontage,lot_area,street,alley,lot_shape,land_contour,utilities,lot_config,land_slope,neighborhood,condition_1,condition_2,bldg_type,house_style,overall_qual,overall_cond,year_built,year_remod/add,roof_style,roof_matl,exterior_1st,exterior_2nd,mas_vnr_type,mas_vnr_area,exter_qual,exter_cond,foundation,bsmt_qual,bsmt_cond,bsmt_exposure,bsmtfin_type_1,bsmtfin_sf_1,bsmtfin_type_2,bsmtfin_sf_2,bsmt_unf_sf,total_bsmt_sf,heating,heating_qc,central_air,electrical,1st_flr_sf,2nd_flr_sf,low_qual_fin_sf,gr_liv_area,bsmt_full_bath,bsmt_half_bath,full_bath,half_bath,bedroom_abvgr,kitchen_abvgr,kitchen_qual,totrms_abvgrd,functional,fireplaces,fireplace_qu,garage_type,garage_yr_blt,garage_finish,garage_cars,garage_area,garage_qual,garage_cond,paved_drive,wood_deck_sf,open_porch_sf,enclosed_porch,3ssn_porch,screen_porch,pool_area,pool_qc,fence,misc_feature,misc_val,mo_sold,yr_sold,sale_type,saleprice
52,2351,527356020,60,RL,80.0,16692,Pave,None,IR1,Lvl,AllPub,Inside,Gtl,NWAmes,RRAn,Norm,1Fam,2Story,7,5,1978,1978,Gable,CompShg,Plywood,Plywood,BrkFace,184.0,TA,TA,CBlock,Gd,TA,No,BLQ,790.0,LwQ,469.0,133.0,1392.0,GasA,TA,Y,SBrkr,1392,1392,0,2784,1.0,0.0,3,1,5,1,Gd,12,Typ,2,TA,Attchd,1978.0,RFn,2.0,564.0,TA,TA,Y,0,112,0,0,440,519,Fa,MnPrv,TenC,2000,7,2006,WD,250000
113,166,535457010,20,RL,87.0,10000,Pave,None,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,6,1962,1962,Hip,CompShg,Wd Sdng,Wd Sdng,BrkFace,261.0,TA,TA,CBlock,TA,TA,No,Unf,0.0,Unf,0.0,1116.0,1116.0,GasA,TA,Y,SBrkr,1116,0,0,1116,0.0,0.0,1,1,3,1,TA,5,Typ,0,No Fireplace,Attchd,1962.0,Unf,2.0,440.0,TA,TA,Y,0,0,0,0,385,0,No Pool,No Fence,No Addl Features,0,2,2010,WD,160000
240,2740,905451050,20,RL,80.0,12048,Pave,None,Reg,Lvl,AllPub,Inside,Gtl,Edwards,Norm,Norm,1Fam,1Story,5,6,1952,2002,Gable,CompShg,Wd Sdng,Wd Sdng,BrkFace,232.0,TA,TA,Slab,No Basement,No Basement,No Basement,No Basement,0.0,No Basement,0.0,0.0,0.0,GasA,Gd,Y,SBrkr,1488,0,0,1488,0.0,0.0,1,0,3,1,TA,7,Typ,1,Ex,Attchd,2002.0,RFn,2.0,569.0,TA,TA,Y,0,189,36,0,348,0,No Pool,No Fence,No Addl Features,0,4,2006,WD,135000
438,626,535125070,20,RL,80.0,14695,Pave,None,IR1,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,8,1966,2008,Gable,CompShg,MetalSd,MetalSd,BrkFace,210.0,TA,Gd,CBlock,TA,TA,No,ALQ,1387.0,Unf,0.0,175.0,1562.0,GasA,Gd,Y,SBrkr,1567,0,0,1567,1.0,0.0,2,0,2,1,Gd,5,Typ,2,Gd,Attchd,1966.0,Unf,2.0,542.0,TA,TA,Y,0,110,0,0,342,0,No Pool,GdWo,No Addl Features,0,7,2009,WD,256000
703,2863,909280030,50,RL,86.0,11500,Pave,None,IR1,Lvl,AllPub,Inside,Gtl,Crawfor,Norm,Norm,1Fam,1.5Fin,7,7,1936,1987,Gable,CompShg,BrkFace,BrkFace,None,0.0,Gd,TA,CBlock,Gd,TA,No,Rec,223.0,Unf,0.0,794.0,1017.0,GasA,Gd,Y,SBrkr,1020,1037,0,2057,0.0,0.0,1,1,3,1,Gd,6,Typ,1,Gd,Attchd,1936.0,Fin,1.0,180.0,Fa,TA,Y,0,0,0,0,322,0,No Pool,No Fence,No Addl Features,0,6,2006,WD,250000
974,1289,902105020,50,RM,60.0,10440,Pave,Grvl,Reg,Lvl,AllPub,Corner,Gtl,OldTown,Norm,Norm,1Fam,1.5Fin,6,7,1920,1950,Gable,CompShg,BrkFace,Wd Sdng,None,0.0,Gd,Gd,BrkTil,Gd,TA,No,LwQ,493.0,Unf,0.0,1017.0,1510.0,GasW,Ex,Y,SBrkr,1584,1208,0,2792,0.0,0.0,2,0,5,1,TA,8,Mod,2,TA,Detchd,1920.0,Unf,2.0,520.0,Fa,TA,Y,0,547,0,0,480,0,No Pool,MnPrv,Shed,1150,6,2008,WD,256000
1035,2667,902400110,75,RM,90.0,22950,Pave,None,IR2,Lvl,AllPub,Inside,Gtl,OldTown,Artery,Norm,1Fam,2.5Fin,10,9,1892,1993,Gable,WdShngl,Wd Sdng,Wd Sdng,None,0.0,Gd,Gd,BrkTil,TA,TA,Mn,Unf,0.0,Unf,0.0,1107.0,1107.0,GasA,Ex,Y,SBrkr,1518,1518,572,3608,0.0,0.0,2,1,4,1,Ex,12,Typ,2,TA,Detchd,1993.0,Unf,3.0,840.0,Ex,TA,Y,0,260,0,0,410,0,No Pool,GdPrv,No Addl Features,0,6,2006,WD,475000
1174,2722,905200340,20,RL,102.0,17920,Pave,None,Reg,Lvl,AllPub,Inside,Gtl,Sawyer,Norm,Norm,1Fam,1Story,5,4,1955,1974,Hip,CompShg,Wd Sdng,Plywood,None,0.0,TA,TA,CBlock,TA,TA,Mn,ALQ,306.0,Rec,1085.0,372.0,1763.0,GasA,TA,Y,SBrkr,1779,0,0,1779,1.0,0.0,1,1,3,1,TA,6,Typ,1,Gd,Attchd,1955.0,Unf,2.0,454.0,TA,TA,Y,0,418,0,0,312,0,No Pool,No Fence,No Addl Features,0,7,2006,WD,170000
1357,388,527375100,20,RL,0.0,9373,Pave,Non

Again, nothing makes me think this is unreasonable.

In [101]:
df[df['kitchen_abvgr']==3]

,id,pid,ms_subclass,ms_zoning,lot_frontage,lot_area,street,alley,lot_shape,land_contour,utilities,lot_config,land_slope,neighborhood,condition_1,condition_2,bldg_type,house_style,overall_qual,overall_cond,year_built,year_remod/add,roof_style,roof_matl,exterior_1st,exterior_2nd,mas_vnr_type,mas_vnr_area,exter_qual,exter_cond,foundation,bsmt_qual,bsmt_cond,bsmt_exposure,bsmtfin_type_1,bsmtfin_sf_1,bsmtfin_type_2,bsmtfin_sf_2,bsmt_unf_sf,total_bsmt_sf,heating,heating_qc,central_air,electrical,1st_flr_sf,2nd_flr_sf,low_qual_fin_sf,gr_liv_area,bsmt_full_bath,bsmt_half_bath,full_bath,half_bath,bedroom_abvgr,kitchen_abvgr,kitchen_qual,totrms_abvgrd,functional,fireplaces,fireplace_qu,garage_type,garage_yr_blt,garage_finish,garage_cars,garage_area,garage_qual,garage_cond,paved_drive,wood_deck_sf,open_porch_sf,enclosed_porch,3ssn_porch,screen_porch,pool_area,pool_qc,fence,misc_feature,misc_val,mo_sold,yr_sold,sale_type,saleprice
1359,716,902325070,75,RM,90.0,8100,Pave,None,Reg,Lvl,AllPub,Corner,Gtl,OldTown,Norm,Norm,1Fam,2.5Unf,5,5,1898,1965,Hip,CompShg,AsbShng,AsbShng,None,0.0,TA,TA,PConc,TA,TA,No,Unf,0.0,Unf,0.0,849.0,849.0,GasA,TA,N,FuseA,1075,1063,0,2138,0.0,0.0,2,0,2,3,TA,11,Typ,0,No Fireplace,Detchd,1910.0,Unf,2.0,360.0,Fa,Po,N,40,156,0,0,0,0,No Pool,MnPrv,No Addl Features,0,11,2009,WD,106000


In [106]:
df[df['kitchen_abvgr']==2].shape

(87, 81)

I don't see anything in the 3 kitchen house to make me think that it isn't real, so I'm going to change it to the average kitchen number, 1.

In [156]:
df2['kitchen_abvgr'].mean()

1.043026706231454

In [158]:
df2.loc[df2[df2['kitchen_abvgr']==3].index, 'kitchen_abvgr'] = 1

In [159]:
df2[df2['kitchen_abvgr'] == 3]

,id,pid,ms_subclass,ms_zoning,lot_frontage,lot_area,street,alley,lot_shape,land_contour,utilities,lot_config,land_slope,neighborhood,condition_1,condition_2,bldg_type,house_style,overall_qual,overall_cond,year_built,year_remod/add,roof_style,roof_matl,exterior_1st,exterior_2nd,mas_vnr_type,mas_vnr_area,exter_qual,exter_cond,foundation,bsmt_qual,bsmt_cond,bsmt_exposure,bsmtfin_type_1,bsmtfin_sf_1,bsmtfin_type_2,bsmtfin_sf_2,bsmt_unf_sf,total_bsmt_sf,heating,heating_qc,central_air,electrical,1st_flr_sf,2nd_flr_sf,low_qual_fin_sf,gr_liv_area,bsmt_full_bath,bsmt_half_bath,full_bath,half_bath,bedroom_abvgr,kitchen_abvgr,kitchen_qual,totrms_abvgrd,functional,fireplaces,fireplace_qu,garage_type,garage_yr_blt,garage_finish,garage_cars,garage_area,garage_qual,garage_cond,paved_drive,wood_deck_sf,open_porch_sf,enclosed_porch,3ssn_porch,screen_porch,pool_area,pool_qc,fence,misc_feature,misc_val,mo_sold,yr_sold,sale_type,saleprice


In [165]:
# As noted above, I decided not to use the dataframe from that resulted from the additional cleaning,
# as the predictions that resulted were a bit weaker.

# df2.to_csv('datasets/draft3_cleaned_train.csv', index = False)